In [1]:
%load_ext autotime

In [2]:
import boto3
import logging
import json
import pickle
import pytest
import sys
import time

from etltools import s3

reload(logging)  # get around notebook problem

<module 'logging' from '/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/logging/__init__.pyc'>

time: 585 ms


In [3]:
logging.basicConfig(
    level=logging.INFO, 
    format='[%(asctime)s] {%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
#     handlers=[
#         logging.FileHandler(filename='mylog.log', mode='w'),
#         logging.StreamHandler(sys.stdout),
#     ]
)

time: 1.55 ms


In [4]:
# Test whether logging works:
logger = logging.getLogger()
logger.info('TEST INFO')

[2018-01-07 10:49:39,386] {<ipython-input-4-efd2929c73d3>:3} INFO - TEST INFO


time: 1.69 ms


In [5]:
aws_info = {
    'profile_name': 'sandbox',
}

uids = s3.read_json('s3://picwell.sandbox.medicare/samples/philadelphia-2015-1k-sample')
pids = ['2820028008119', '2820088001036']

time: 279 ms


# Test ConfigInfo

In [6]:
from lambda_client.config_info import ConfigInfo

configs = ConfigInfo('lambda_client/lambda.cfg')

print configs.claims_bucket
print configs.claims_path
print
print configs.benefits_bucket
print configs.benefits_path
print
print configs.claims_table

picwell.sandbox.analytics
junghoon/lambda_calculator

picwell.sandbox.analytics
junghoon/lambda_calculator_benefits

ma_claims
time: 4.71 ms


In [7]:
all_states = configs.all_states

print '{} states'.format(len(all_states))
print all_states

51 states
['01', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56', '72']
time: 1.14 ms


# Test ClaimsClient

In [8]:
from lambda_client import ClaimsClient

time: 569 µs


In [9]:
# Test S3:
client = ClaimsClient(aws_info, 
                      s3_bucket=configs.claims_bucket,
                      s3_path=configs.claims_path)

client.get(uids[0])

{u'medical_claims': [{u'admitted': u'2014-04-03',
   u'benefit_category': 16,
   u'cost': u'148.0',
   u'discharged': u'2014-04-03',
   u'length_of_stay': 1},
  {u'admitted': u'2014-05-02',
   u'benefit_category': 16,
   u'cost': u'74.55',
   u'discharged': u'2014-05-02',
   u'length_of_stay': 1},
  {u'admitted': u'2014-05-05',
   u'benefit_category': 16,
   u'cost': u'104.39',
   u'discharged': u'2014-05-05',
   u'length_of_stay': 1},
  {u'admitted': u'2014-05-13',
   u'benefit_category': 16,
   u'cost': u'210.12',
   u'discharged': u'2014-05-13',
   u'length_of_stay': 1},
  {u'admitted': u'2014-05-19',
   u'benefit_category': 11,
   u'cost': u'442.2',
   u'discharged': u'2014-05-19',
   u'length_of_stay': 1},
  {u'admitted': u'2014-06-10',
   u'benefit_category': 11,
   u'cost': u'16.92',
   u'discharged': u'2014-06-10',
   u'length_of_stay': 1},
  {u'admitted': u'2014-06-10',
   u'benefit_category': 16,
   u'cost': u'207.11',
   u'discharged': u'2014-06-10',
   u'length_of_stay': 1}

time: 243 ms


In [10]:
# Test DynamoDB:
client = ClaimsClient(aws_info,
                      table_name=configs.claims_table)

client.get(uids[0])

{u'medical_claims': [{u'admitted': u'2014-04-03',
   u'benefit_category': Decimal('16'),
   u'cost': u'148.0',
   u'discharged': u'2014-04-03',
   u'length_of_stay': Decimal('1')},
  {u'admitted': u'2014-05-02',
   u'benefit_category': Decimal('16'),
   u'cost': u'74.55',
   u'discharged': u'2014-05-02',
   u'length_of_stay': Decimal('1')},
  {u'admitted': u'2014-05-05',
   u'benefit_category': Decimal('16'),
   u'cost': u'104.39',
   u'discharged': u'2014-05-05',
   u'length_of_stay': Decimal('1')},
  {u'admitted': u'2014-05-13',
   u'benefit_category': Decimal('16'),
   u'cost': u'210.12',
   u'discharged': u'2014-05-13',
   u'length_of_stay': Decimal('1')},
  {u'admitted': u'2014-05-19',
   u'benefit_category': Decimal('11'),
   u'cost': u'442.2',
   u'discharged': u'2014-05-19',
   u'length_of_stay': Decimal('1')},
  {u'admitted': u'2014-06-10',
   u'benefit_category': Decimal('11'),
   u'cost': u'16.92',
   u'discharged': u'2014-06-10',
   u'length_of_stay': Decimal('1')},
  {u'ad

time: 243 ms


In [11]:
# Test configuration file:
client = ClaimsClient(aws_info)

client.get(uids[0])

{u'medical_claims': [{u'admitted': u'2014-04-03',
   u'benefit_category': 16,
   u'cost': u'148.0',
   u'discharged': u'2014-04-03',
   u'length_of_stay': 1},
  {u'admitted': u'2014-05-02',
   u'benefit_category': 16,
   u'cost': u'74.55',
   u'discharged': u'2014-05-02',
   u'length_of_stay': 1},
  {u'admitted': u'2014-05-05',
   u'benefit_category': 16,
   u'cost': u'104.39',
   u'discharged': u'2014-05-05',
   u'length_of_stay': 1},
  {u'admitted': u'2014-05-13',
   u'benefit_category': 16,
   u'cost': u'210.12',
   u'discharged': u'2014-05-13',
   u'length_of_stay': 1},
  {u'admitted': u'2014-05-19',
   u'benefit_category': 11,
   u'cost': u'442.2',
   u'discharged': u'2014-05-19',
   u'length_of_stay': 1},
  {u'admitted': u'2014-06-10',
   u'benefit_category': 11,
   u'cost': u'16.92',
   u'discharged': u'2014-06-10',
   u'length_of_stay': 1},
  {u'admitted': u'2014-06-10',
   u'benefit_category': 16,
   u'cost': u'207.11',
   u'discharged': u'2014-06-10',
   u'length_of_stay': 1}

time: 214 ms


In [12]:
# The object should not be pickled.
with pytest.raises(Exception, match='ClaimsClient object cannot be pickled.'):
    pickle.dumps(client)

time: 1.82 ms


# Test BenefitsClient

In [13]:
from lambda_client import BenefitsClient

time: 1.16 ms


In [14]:
client = BenefitsClient(aws_info)

print client.all_states

['01', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56', '72']
time: 1.24 ms


In [15]:
plans = client._get_one_state('01')
print '{} plans read for state 01'.format(len(plans))

plans = client._get_one_state('04')
print '{} plans read for state 04'.format(len(plans))

47 plans read for state 01
75 plans read for state 04
time: 799 ms


In [16]:
plans = client.get_by_state(['01', '04'])
print '{} plans read'.format(len(plans))

[2018-01-07 10:49:41,216] {storage_utils.py:156} INFO - 0.001442 seconds to start all threads for get_by_state().
[2018-01-07 10:49:41,749] {storage_utils.py:165} INFO - 0.529701 seconds to join all threads for get_by_state().
[2018-01-07 10:49:41,749] {storage_utils.py:175} INFO - 3.3e-05 seconds to combine results for get_by_state().


122 plans read
time: 538 ms


In [17]:
plans = client.get_all()
print '{} plans read'.format(len(plans))

[2018-01-07 10:49:41,929] {storage_utils.py:156} INFO - 0.172866 seconds to start all threads for get_by_state().
[2018-01-07 10:49:51,061] {storage_utils.py:165} INFO - 9.126392 seconds to join all threads for get_by_state().
[2018-01-07 10:49:51,062] {storage_utils.py:175} INFO - 0.000423 seconds to combine results for get_by_state().


3558 plans read
time: 9.31 s


In [18]:
# Compare the timing against reading the entire file:
from lambda_client.storage_utils import _read_json

session = boto3.Session(**aws_info)
resource = session.resource('s3')

time: 65.8 ms


In [19]:
all_plans = _read_json('picwell.sandbox.medicare', 'ma_benefits/cms_2018_pbps_20171005.json', resource)

print '{} plans read'.format(len(plans))

3558 plans read
time: 3.98 s


In [20]:
# Ensure that the same plans are read:
sort_key = lambda plan: plan['picwell_id']
assert sorted(all_plans, key=sort_key) == sorted(plans, key=sort_key)

time: 143 ms


In [21]:
# The object should not be pickled.
with pytest.raises(Exception, match='BenefitsClient object cannot be pickled.'):
    pickle.dumps(client)

time: 1.85 ms


# Test Cost Breakdown

In [22]:
from lambda_client import CalculatorClient

client = CalculatorClient(aws_info)

time: 993 µs


In [23]:
responses = client._get_one_breakdown(uids[0], pids, '01')

print '{} responses returned'.format(len(responses))
responses[0]

2 responses returned


{u'allowed': 26376.640000000003,
 u'covered_breakdown': {u'categories': {u'0': 0.0,
   u'11': 186.33999999999997,
   u'13': 13.23,
   u'15': 30.79,
   u'16': 0.0,
   u'19': 80.0,
   u'25': 1750.0,
   u'30': 719.1700000000001,
   u'31': 300.0,
   u'44': 0.0,
   u'49': 118.48800000000001,
   u'7': 479.558},
  u'composite': 3677.5759999999996,
  u'in_network': 3677.5759999999996,
  u'out_network': 0.0},
 u'deductible_breakdown': {u'categories': {u'0': 0.0,
   u'11': 0.0,
   u'13': 0.0,
   u'15': 0.0,
   u'16': 0.0,
   u'19': 0.0,
   u'25': 0.0,
   u'30': 0.0,
   u'31': 0.0,
   u'44': 0.0,
   u'49': 0.0,
   u'7': 0.0},
  u'composite': 0.0,
  u'in_network': 0.0,
  u'out_network': 0.0},
 u'oop': 3677.5759999999996,
 u'picwell_id': u'2820028008119',
 u'uid': u'1302895801',
 u'uncovered': 0.0,
 u'uncovered_breakdown': {u'categories': {u'0': 0.0,
   u'11': 0.0,
   u'13': 0.0,
   u'15': 0.0,
   u'16': 0.0,
   u'19': 0.0,
   u'25': 0.0,
   u'30': 0.0,
   u'31': 0.0,
   u'44': 0.0,
   u'49': 0.0,


time: 5.28 s


In [24]:
responses = client.get_breakdown(uids[:3], pids)

print '{} responses returned'.format(len(responses))

[2018-01-07 10:50:00,613] {calculator_client.py:79} INFO - 0.053427 seconds to start all threads for get_breakdown().
[2018-01-07 10:50:06,260] {calculator_client.py:88} INFO - 5.646606 seconds to join all threads for get_breakdown().
[2018-01-07 10:50:06,261] {calculator_client.py:98} INFO - 2.5e-05 seconds to combine results for get_breakdown().


6 responses returned
time: 5.7 s


In [25]:
# Let's try something larger:
responses = client.get_breakdown(uids, pids)

print '{} responses returned'.format(len(responses))

[2018-01-07 10:51:20,357] {calculator_client.py:79} INFO - 74.090978 seconds to start all threads for get_breakdown().
[2018-01-07 10:51:31,042] {calculator_client.py:88} INFO - 10.634157 seconds to join all threads for get_breakdown().
[2018-01-07 10:51:31,048] {calculator_client.py:98} INFO - 0.004769 seconds to combine results for get_breakdown().


2000 responses returned
time: 1min 24s


# Test Batch Calculation

In [26]:
# uids = s3.read_json('s3n://picwell.sandbox.medicare/samples/philadelphia-2015')

# print '{} uids read'.format(len(uids))

time: 866 µs


In [27]:
# uids[:10]

time: 825 µs


In [28]:
# requests_per_second = 100

# for uid in uids:
# #     client.calculate_async(uid, months=['01'])
#     client.calculate_async(uid)
#     time.sleep(1.0/requests_per_second)  

time: 1.15 ms
